# Analysis

In [ ]:
%reset -f

# Genutzte Umgebung

In [ ]:
import pandas as pd
from logger import Logger
from test_executer import TestExecutor
import extract
import sys
import numpy as np

logger = Logger()
testExecutor = TestExecutor(logger)

dependencies = [
    ('system', sys.version_info),
    ('pandas', pd.__version__),
    ('numpy', np.__version__),
]
for dependency in dependencies:
    logger.log(f"{dependency[0]} is installed with version {dependency[1]}")

## Setup der Daten

Zuerst laden wir die benötigten Daten herunter und initialisieren die genutzten Python Objekte.

In [ ]:
tables = [
    "careplans",
    "conditions",
    "observations",
    "patients",
]

files = [
    "data/others/",
    "data/asthma/",
    "data/gallstones/",
    "data/hypertension/",
]

In [ ]:
!mkdir -p data/allergy

In [ ]:
from urllib.request import urlopen
import os

def ensure_file_has_been_downloaded(filename):
    full_filename = "../" + filename

    url = "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/" + filename
    if os.path.isfile(full_filename):
        logger.log("File {} already exists, skipping download".format(filename))
    else:
        logger.log("Downloading {}".format(filename))
        download_file(url, full_filename)

def download_file(url, filename):
    with open(filename, 'wb') as out_file:
        with urlopen(url) as file:
            out_file.write(file.read())

if not os.path.isfile("extract.py"):
    download_file(
        "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/src/extract.py",
        "extract.py"
    )

for file in files:
    for table in tables:
        ensure_file_has_been_downloaded(file+table+".csv")

## Mit Datenbank verbinden

In [ ]:
databaseFile = "data.sqlite"

logger.startTimeMeasurement('open-db', 'Connected to db and created tables')
connection = extract.connect_to_db(logger, databaseFile)  # create table patients, observations, conditions, careplans
logger.endTimeMeasurement('open-db')

In [ ]:
def test_sqliteConnection(_logger):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablesInDb = list(map(lambda tableResult: tableResult[0], cursor.fetchall()))
    tablesInDb.sort()

    for table in tables:
        if not(table in tablesInDb):
            raise Exception('Table not found:', table)

testExecutor.execute('Test connection to database', test_sqliteConnection)

## Daten in Datenbank laden

Lade der verwendete Daten in die Datenbank:

-   careplans
-   conditions
-   observations
-   patients

In [ ]:
logger.startTimeMeasurement('load-data', 'Loading data into db')
for file in files:
    for table in tables:
        extract.insert_values_to_table(logger, connection.cursor(), table, "../"+ file + table + ".csv")  # TODO: insert ALL values in the right tables 
        connection.commit()

logger.endTimeMeasurement('load-data')

## Messung der Datenfehler

Für unsere Forschungsfrage sind nur alle Daten mit gemessenen BMI relevant. Wurde dieser nicht vermessen oder eingetragen können die Daten für die Forschungsfrage nicht verwendet werden und sind somit unbrauchbar.

In [ ]:
all_patients_query = """
SELECT COUNT(id) FROM patients;"""
count_bmi_query = """
SELECT COUNT(distinct id) FROM patients JOIN observations on patients.id == observations.patient WHERE observations.Code = '59576-9'"""

count_all_bmi_query = f"""
SELECT COUNT(patient) FROM observations WHERE observations.Code = '59576-9'"""

patient_all_count = connection.execute(all_patients_query).fetchall()[0][0]
patient_bmi_count = connection.execute(count_bmi_query).fetchall()[0][0]
bmi_count = connection.execute(count_all_bmi_query).fetchall()[0][0]
ratio = round(patient_bmi_count/patient_all_count, 3) * 100

logger.log(f"Total num of patients {patient_all_count}.")
logger.log(f"Found {patient_bmi_count} patients ({ratio}%) with {bmi_count} BMI-values.")

In [ ]:
cursor = connection.cursor()

In [ ]:
# print tables of the database
cursor.execute('SELECT name FROM sqlite_master where type="table"') 
print(cursor.fetchall())

# Star Schema

#### Erstellen Dimensionstabellen

Tabelle patients_

In [ ]:
# table patients_
cursor.execute('''DROP TABLE IF EXISTS patients_;''')
cursor.execute('''
        CREATE TABLE patients_ ( 
        ID STRING PRIMARY KEY UNIQUE,
        RACE STRING, 
        ETHNICITY STRING,
        GENDER STRING,
        AGE INT64
        );''')

In [ ]:
# create df patients
cursor.execute('''SELECT ID, BIRTHDATE, DEATHDATE, RACE, ETHNICITY, GENDER FROM PATIENTS;''')
df_patients = pd.DataFrame(cursor.fetchall(), columns=['ID', 'BIRTHDATE', 'DEATHDATE', 'RACE', 'ETHNICITY', 'GENDER'])

In [ ]:
# convert to date
df_patients["DEATHDATE"] = pd.to_datetime(df_patients["DEATHDATE"])
df_patients["BIRTHDATE"] = pd.to_datetime(df_patients["BIRTHDATE"])
# fill null values withh todays date
df_patients['DEATHDATE'] = df_patients.DEATHDATE.fillna(pd.to_datetime("today"))
# calculate age
df_patients["AGE"] = df_patients.DEATHDATE.dt.year - df_patients.BIRTHDATE.dt.year
# drop unnecessary variables
df_patients = df_patients.drop(['BIRTHDATE', 'DEATHDATE'], axis=1)

In [ ]:
df_patients.info()

In [ ]:
print("Number of Duplicated Rows", df_patients.duplicated(df_patients.columns).sum())

In [ ]:
df_patients.to_sql('df_patients', connection, if_exists='replace', index=False)
cursor.execute('INSERT INTO patients_ (ID, RACE, ETHNICITY, GENDER, AGE) SELECT ID, RACE, ETHNICITY, GENDER, RACE FROM df_patients;')
cursor.execute('''DROP TABLE IF EXISTS df_patients;''')

print(pd.read_sql_query("PRAGMA table_info(patients_)", connection))

Tabelle observations_

In [ ]:
# table observations_
cursor.execute('''DROP TABLE IF EXISTS observations_;''')
cursor.execute('''
        CREATE TABLE observations_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING,
        UNITS STRING, 
        TYPE STRING
        );''')

# create df observation
cursor.execute('''SELECT CODE, DESCRIPTION, UNITS, TYPE FROM OBSERVATIONS;''')
df_observations = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION', 'UNITS', 'TYPE'])

df_observations = df_observations.drop_duplicates(subset='CODE')

# transform dt in table
df_observations.to_sql('df_observations', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO observations_ (CODE, DESCRIPTION, UNITS, TYPE) SELECT CODE, DESCRIPTION, UNITS, TYPE FROM df_observations;''')

cursor.execute('''DROP TABLE IF EXISTS df_observations;''')
print(pd.read_sql_query("PRAGMA table_info(observations_)", connection))
#print(pd.read_sql_query("SELECT * FROM observations_", connection))

Tabelle careplans_code

In [ ]:
# table careplans_code
cursor.execute('''DROP TABLE IF EXISTS careplans_code;''')
cursor.execute('''
        CREATE TABLE careplans_code ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT CODE, DESCRIPTION FROM CAREPLANS;''')
df_careplans_code = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_careplans_code = df_careplans_code.drop_duplicates(subset='CODE')

# transform dt in table
df_careplans_code.to_sql('df_careplans_code', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO careplans_code (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_careplans_code;''')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_code;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_code)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_code", connection))

Tabelle careplans_reasoncode

In [ ]:
# table careplans_reasoncode
cursor.execute('''DROP TABLE IF EXISTS careplans_reasoncode;''')
cursor.execute('''
        CREATE TABLE careplans_reasoncode ( 
        REASONCODE STRING PRIMARY KEY UNIQUE,
        REASONDESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT REASONCODE, REASONDESCRIPTION FROM CAREPLANS;''')
df_careplans_reasoncode = pd.DataFrame(cursor.fetchall(), columns=['REASONCODE','REASONDESCRIPTION'])

df_careplans_reasoncode = df_careplans_reasoncode.drop_duplicates(subset='REASONCODE')

# transform dt in table
df_careplans_reasoncode.to_sql('df_careplans_reasoncode', connection, if_exists='replace', index=False)
   
cursor.execute('INSERT INTO careplans_reasoncode (REASONCODE, REASONDESCRIPTION) SELECT REASONCODE, REASONDESCRIPTION FROM df_careplans_reasoncode;')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_reasoncode;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_reasoncode)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_reasoncode", connection))

Tabelle conditions_

In [ ]:
# table conditions
cursor.execute('''DROP TABLE IF EXISTS conditions_;''')
cursor.execute('''
        CREATE TABLE conditions_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df conditions
cursor.execute('''SELECT CODE, DESCRIPTION FROM CONDITIONS;''')
df_conditions = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_conditions = df_conditions.drop_duplicates(subset='CODE')

# transform dt in table
df_conditions.to_sql('df_conditions', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO conditions_ (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_conditions;')

cursor.execute('''DROP TABLE IF EXISTS df_conditions;''')
print(pd.read_sql_query("PRAGMA table_info(conditions_)", connection))
#print(pd.read_sql_query("SELECT * FROM conditions_", connection))

#### Erstelle Faktentabelle

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS fact_table;''')
cursor.execute('''
        CREATE TABLE fact_table ( 
        PATIENT_ID STRING,
        OBSERVATION_CODE STRING,
        VALUE STRING,
        DATE DATE,
        CONDITIONS_CODE STRING,
        ENDDATE DATE,
        CAREPLANS_CODE STRING,
        CAREPLANS_REASONCODE STRING,
        FOREIGN KEY (PATIENT_ID)
            REFERENCES patients_(ID),
        FOREIGN KEY (OBSERVATION_CODE)
            REFERENCES observations_(CODE),
        FOREIGN KEY (CONDITIONS_CODE)
            REFERENCES conditions_(CODE),
        FOREIGN KEY (CAREPLANS_CODE)
            REFERENCES careplans_code(CODE),
        FOREIGN KEY (CAREPLANS_REASONCODE)
            REFERENCES careplans_reasoncode(REASONCODE)
        );''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, OBSERVATION_CODE, VALUE, DATE) 
                    SELECT PATIENT, CODE, VALUE, DATE 
                    FROM OBSERVATIONS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_REASONCODE, DATE, ENDDATE) 
                    SELECT PATIENT, REASONCODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CONDITIONS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CONDITIONS
                    ;''')

In [ ]:
print(pd.read_sql_query("PRAGMA foreign_key_list(fact_table)", connection))

In [ ]:
cursor.execute('SELECT name FROM sqlite_master where type="table"')
print(cursor.fetchall())

## Analysis

## Aufräumen & Logs speichern

In [ ]:
connection.close()
logger.logTimings()
logger.writeToFile("../artefacts-for-release/analysis-log.json")